In [5]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_algorithms.optimizers import SLSQP
from qiskit_algorithms.minimum_eigensolvers import AdaptVQE, VQE
from qiskit.primitives import Estimator

info = MoleculeInfo(["Li", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, 1.5)])
driver = PySCFDriver.from_molecule(info, basis="sto3g")
molecule = driver.run()

molecule = FreezeCoreTransformer().transform(molecule)
hamiltonian = molecule.hamiltonian.second_q_op()
mapper = ParityMapper(num_particles=molecule.num_particles)
tapered_mapper = molecule.get_tapered_mapper(mapper)
qubit_op = tapered_mapper.map(hamiltonian)

init_state = HartreeFock(
    molecule.num_spatial_orbitals,
    molecule.num_particles,
    tapered_mapper
)

adapt_vqe_ansatz = UCCSD(
    molecule.num_spatial_orbitals,
    molecule.num_particles,
    tapered_mapper,
    initial_state=init_state
)

optimizer = SLSQP(maxiter=10000, ftol=1e-9)
adapt_vqe = AdaptVQE(VQE(Estimator(), adapt_vqe_ansatz, optimizer))

result_adapt_vqe = adapt_vqe.compute_minimum_eigenvalue(qubit_op)
print("Adapt-VQE Energy:", result_adapt_vqe.eigenvalue)


/tmp/ipykernel_749345/1372990882.py:34: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  adapt_vqe = AdaptVQE(VQE(Estimator(), adapt_vqe_ansatz, optimizer))


Adapt-VQE Energy: -1.100179536751887
